# Calculo de $\pi$ secuencial

In [1]:
%%writefile phiseccollab.c

#include <stdio.h>
#include <stdlib.h>
#include <sys/time.h>

#define iterations 1e9
double pi=0;

double pileibnitz(){
    for(int i=0; i< iterations; i++){
        pi += (double) 1/(i*2+1) * 4;
        i++;
        pi -= (double) 1/(i*2+1) * 4 ;
    }
    return pi;
}

int main(){
    FILE *fo;
    fo = fopen("pithreaddataCOLLAB.txt", "a");
    struct timeval t_before, t_after, t_execution; 
    gettimeofday(&t_before, NULL);
    double pi = pileibnitz();
    gettimeofday(&t_after, NULL);
    timersub(&t_after, &t_before, &t_execution);
    long int seconds = (long int) t_execution.tv_sec;
    long int useconds = (long int) t_execution.tv_usec;
    fprintf(fo, "1Thread %ld.%06ld", seconds, useconds);
    fclose(fo);
    printf("Pi calculated with %f iteractions \n With a value of %f \n Executed in %ld.%06ld seconds", iterations, pi, seconds, useconds);
    return 0;
}

Writing phiseccollab.c


In [2]:
%%shell 
gcc phiseccollab.c -o output
./output

Pi calculated with 1000000000.000000 iteractions 
 With a value of 3.141593 
 Executed in 11.111123 seconds

# Calculo de $\pi$ 2 Hilos

In [3]:
%%writefile phi2thcollab.c

#include <stdio.h>
#include <stdlib.h>
#include <sys/time.h>
#include <pthread.h>

#define iterations 1e9
#define threads 2

double pi=0;
int section = iterations/threads;

void *pileibnitz(void *arg){
    int hilo = (int *) arg;
    int init = section * hilo;
    int fn = init + section -1;
    for(int init=0; init < fn; init++){
        pi += (double) 1/(init*2+1) * 4;
        init++;
        pi -= (double) 1/(init*2+1) * 4 ;
    }
}

int main(){
    FILE *fo;
    fo = fopen("pithreaddataCOLLAB.txt", "a");
    struct timeval t_before, t_after, t_execution; 
    pthread_t thread1;
    pthread_t thread2;
    gettimeofday(&t_before, NULL);
    pthread_create(&thread1, NULL, (void *)pileibnitz, (void *) 0);
    pthread_create(&thread2, NULL, (void *)pileibnitz, (void *) 1);
    pthread_join(thread1, NULL);
    pthread_join(thread2, NULL);
    gettimeofday(&t_after, NULL);
    timersub(&t_after, &t_before, &t_execution);
    long int seconds = (long int) t_execution.tv_sec;
    long int useconds = (long int) t_execution.tv_usec;
    fprintf(fo, "\n2Thread %ld.%06ld", seconds, useconds);
    fclose(fo);
    printf("Pi calculated with %f iteractions \n With a value of %f \n Executed in %ld.%06ld seconds", iterations, pi, seconds, useconds);
    return 0;
}

Writing phi2thcollab.c


In [4]:
%%shell 
gcc phi2thcollab.c -pthread -o output
./output

phi2thcollab.c: In function ‘pileibnitz’:
phi2thcollab.c:14:16: warning: initialization makes integer from pointer without a cast [-Wint-conversion]
     int hilo = (int *) arg;
                ^
Pi calculated with 1000000000.000000 iteractions 
 With a value of 8.246170 
 Executed in 12.715859 seconds

# Calculo de $\pi$ con 3 hilos

In [5]:
%%writefile phi3thcollab.c

#include <stdio.h>
#include <stdlib.h>
#include <sys/time.h>
#include <pthread.h>

#define iterations 1e9
#define threads 3

double pi=0;
int section = iterations/threads;

void *pileibnitz(void *arg){
    int hilo = (int *) arg;
    int init = section * hilo;
    int fn = init + section -1;
    for(int init=0; init < fn; init++){
        pi += (double) 1/(init*2+1) * 4;
        init++;
        pi -= (double) 1/(init*2+1) * 4 ;
    }
}

int main(){
    FILE *fo;
    fo = fopen("pithreaddataCOLLAB.txt", "a");
    struct timeval t_before, t_after, t_execution; 
    pthread_t thread1;
    pthread_t thread2;
    pthread_t thread3;
    gettimeofday(&t_before, NULL);
    pthread_create(&thread1, NULL, (void *)pileibnitz, (void *) 0);
    pthread_create(&thread2, NULL, (void *)pileibnitz, (void *) 1);
    pthread_create(&thread3, NULL, (void *)pileibnitz, (void *) 2);
    pthread_join(thread1, NULL);
    pthread_join(thread2, NULL);
    pthread_join(thread3, NULL);
    gettimeofday(&t_after, NULL);
    timersub(&t_after, &t_before, &t_execution);
    long int seconds = (long int) t_execution.tv_sec;
    long int useconds = (long int) t_execution.tv_usec;
    fprintf(fo, "\n3Thread %ld.%06ld", seconds, useconds);
    fclose(fo);
    printf("Pi calculated with %f iteractions \n With a value of %f \n Executed in %ld.%06ld seconds", iterations, pi, seconds, useconds);
    return 0;
}

Writing phi3thcollab.c


In [6]:
%%shell 
gcc phi3thcollab.c -pthread -o output
./output

phi3thcollab.c: In function ‘pileibnitz’:
phi3thcollab.c:14:16: warning: initialization makes integer from pointer without a cast [-Wint-conversion]
     int hilo = (int *) arg;
                ^
Pi calculated with 1000000000.000000 iteractions 
 With a value of 15.012631 
 Executed in 16.078208 seconds

# Calculo de $\pi$ con 4 hilos

In [7]:
%%writefile phi4thcollab.c

#include <stdio.h>
#include <stdlib.h>
#include <sys/time.h>
#include <pthread.h>

#define iterations 1e9
#define threads 4

double pi=0;
int section = iterations/threads;

void *pileibnitz(void *arg){
    int hilo = (int *) arg;
    int init = section * hilo;
    int fn = init + section -1;
    for(int init=0; init < fn; init++){
        pi += (double) 1/(init*2+1) * 4;
        init++;
        pi -= (double) 1/(init*2+1) * 4 ;
    }
}

int main(){
    FILE *fo;
    fo = fopen("pithreaddataCOLLAB.txt", "a");
    struct timeval t_before, t_after, t_execution; 
    pthread_t thread1;
    pthread_t thread2;
    pthread_t thread3;
    pthread_t thread4;
    gettimeofday(&t_before, NULL);
    pthread_create(&thread1, NULL, (void *)pileibnitz, (void *) 0);
    pthread_create(&thread2, NULL, (void *)pileibnitz, (void *) 1);
    pthread_create(&thread3, NULL, (void *)pileibnitz, (void *) 2);
    pthread_create(&thread4, NULL, (void *)pileibnitz, (void *) 3);
    pthread_join(thread1, NULL);
    pthread_join(thread2, NULL);
    pthread_join(thread3, NULL);
    pthread_join(thread4, NULL);
    gettimeofday(&t_after, NULL);
    timersub(&t_after, &t_before, &t_execution);
    long int seconds = (long int) t_execution.tv_sec;
    long int useconds = (long int) t_execution.tv_usec;
    fprintf(fo, "\n4Thread %ld.%06ld", seconds, useconds);
    fclose(fo);
    printf("Pi calculated with %f iteractions \n With a value of %f \n Executed in %ld.%06ld seconds", iterations, pi, seconds, useconds);
    return 0;
}

Writing phi4thcollab.c


In [8]:
%%shell 
gcc phi4thcollab.c -pthread -o output
./output

phi4thcollab.c: In function ‘pileibnitz’:
phi4thcollab.c:14:16: warning: initialization makes integer from pointer without a cast [-Wint-conversion]
     int hilo = (int *) arg;
                ^
Pi calculated with 1000000000.000000 iteractions 
 With a value of 22.480368 
 Executed in 19.644783 seconds

In [9]:
! uname -a

Linux ebdf8706ffb0 5.4.144+ #1 SMP Tue Dec 7 09:58:10 PST 2021 x86_64 x86_64 x86_64 GNU/Linux


In [12]:
! lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               79
Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:            0
CPU MHz:             2199.998
BogoMIPS:            4399.99
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            56320K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_sin